# Logistic regression (W2), validation, ROC curves (W3)

This notebook will help you familiarize yourself with the logistics regression.

We will build a classifier based on logistic regression. His task will be to determine the probability of admitting a candidate based on the results of two matura exams (each scaled to 0-100%): in mathematics and biology.

You will need to complete the function codes:
* sigmoida
* cost function
* prediction
* functionCostageReg

Before we get to the right tasks, we need to import the necessary modules:

In [ ]:
import matplotlib
% matplotlib notebook
#matplotlib.use('TkAgg')
import numpy as np
import pylab as plt
import scipy.optimize as so
from ipywidgets import FloatProgress
from IPython.display import display

Hypothetical data can be found in the file:

https://brain.fuw.edu.pl/edu/images/d/d8/Reg_log_data1.txt.

Please download them and save in the current directory.

## Loading data

It is always good to start work from getting acquainted with the data. To this end, we should know the data structure:
* The first two columns contain the results of exams,
* the third column contains a label (belonging to a group)

Load the data and let's see the first 10 lines.

In [ ]:
data = np.loadtxt('reg_log_data1.txt',delimiter=',')
print(data[:10,:])

To make it easier to use them, extracting input data from them as 'X' and output as 'Y':

In [ ]:
X = data[:, [0, 1]]
y = data[:, 2]

It's always good to look at data to get a sense of the problem
We draw data:
* yellow symbol means examples from where y = 1 and
* blue ones with y = 0

In [ ]:
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set3)
plt.xlabel('wynik z matematyki')
plt.ylabel('wynik z biologii')

To comfortably apply vector calculations to logistic regression, you need to modify the matrix X by adding it to the left of the columns of ones. They will multiply the $ \theta_0 $ parameter:

In [ ]:
N = len(y) # ilość przykładów w zbiorze uczącym
XX = np.concatenate((np.ones((N,1)), X),axis = 1)

Let's see how the XX matrix looks like now:

In [ ]:
print(XX[:10,:])

## Hypothesis
For the record, _hypothesis_ in logistic regression has the form:

$\qquad$ $h_\theta(x) = \frac{1}{1+\exp(-\theta^Tx)}$.

In the implementation it is good to think about this function like this:

$\qquad$ $h_\theta(x) = \frac{1}{1+f}$.

where: $f = \exp(-\theta^Tx)$

* implement the hypothesis for logistic regression,
* due to the numerical stability of calculations it is good to limit the range of volatility $ f $ np to the range [1e-8, 1e + 8]:

In [ ]:
def hipoteza(x, theta):
    '''ta funkcja zwraca wartość hipotezy dla danego wejścia x i parametrów theta'''
    f = np.exp(...)
    if f < 1e-8:
        f = 1e-8
    if f>1e8:
        f = 1e8 
    h = 1/(1+f)   
    return h
   

Test the function:

In [ ]:
H0 = hipoteza(XX[0,:], np.zeros((3, 1));)
print('wartość hipotezy dla zerowego przykładu i dla początkowej thety: '+ str(H0))

It should go:

```wartość hipotezy dla zerowego przykładu i dla początkowej thety: [ 0.5]```

## Log-credibility function:
* we find the regression parameters by maximizing [log-credibility function](https://brain.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wykład_6#Funkcja_wiarygodno.C5.9Bci):

$\qquad$ $l(\theta) = \log L(\theta) = \sum_{j=1}^m y^{(j)} \log h(x^{(j)}) + (1 - y^{(j)}) \log (1 - h(x^{(j)}))$

In [ ]:
def funkcjaLogWiarygodnosci(theta, X, y):
    '''Ta funkcja oblicza wartość funkcji log-wiarygodności  dla regresji logistycznej
    używając theta jako parametrów oraz X i y jako zbioru uczącego'''
    l=0.0
    # pętla po przykładach ze zbioru uczącego
    for j in range(len(y)): 
        h = hipoteza(...)
        l +=  ...   
    return l   

In this exercise, we will do it using the optimization functions from the module [scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize) .

There are two consequences:
* These functions are adapted to look for the minima of the objective function. We must therefore give them as arguments a minus log-credibility function

In [ ]:
def minusFunkcjaLogWiarygodnosci(theta, X, y):
    return (-1.)*funkcjaLogWiarygodnosci(theta, X, y)

* Some algorithms may run faster if we implement the explicit derivative form:

$\qquad$ $
\begin{array}{lcl}
\frac{\partial}{\partial \theta_i} l(\theta)  =\sum_{j=1}^m (y^{(j)}-h_\theta(x^{(j)}))x_i^{(j)}
\end{array}
$

In [ ]:
def pochodnaLogWiarygodnosci(theta, X, y):
    '''ta funkcja oblicza wartość pochodnej funkcji log-wiarygodności
    dla podaanych wartości theta, X i y'''
    dl_dtheta = np.zeros(len(theta))
    for i in range(len(theta)): # dla i-tej współrzędnej theta
        for j in range(len(y)):  # dodajemy przyczynki od przykładu j-ego 
            h = ...
            dl_dtheta[i] += ...
    return dl_dtheta

In [ ]:
def  minusPochodnaLogWiarygodnosci(theta, X, y):
    return (-1)*pochodnaLogWiarygodnosci(theta, X, y)

Initialize $ \theta $ parameters at 0:

In [ ]:
xDim = XX.shape[1] # rozmiar wejścia rozszerzonego o jedynki
theta0 = np.zeros((xDim, 1));

Let's see what the initial looks like $\theta$:

In [ ]:
print(theta0)

We calculate log-likelihood functions and its derivative for initial data:

In [ ]:
logWiar = funkcjaLogWiarygodnosci(theta0, XX, y)
pochLogWiar = pochodnaLogWiarygodnosci(theta0, XX, y)

print( 'wartość log-wiarygodności dla początkowej thety: '+ str(logWiar))
print( 'pochodna log-wiarygodnosci dla poczatkowej thety: '+ str(pochLogWiar))

If the implementation is correct, the outputs will be:
```
wartość log-wiarygodności dla początkowej thety: [-69.31471806]
pochodna log-wiarygodnosci dla poczatkowej thety: [   10.          1200.92165893  1126.28422055]
```

## Optimization

* Optimization functions are taken from the scipy.optimize module
* Because these functions are implemented to modernize, instead of maximizing the low-credibility function, we will minimize this function multiplied by -1 or minusLog FunctionTrialality fprime = minusLogalLogality,

In [ ]:
theta_opt = so.fmin_bfgs(minusFunkcjaLogWiarygodnosci, theta0, 
                         fprime=minusPochodnaLogWiarygodnosci, 
                         args=(XX,y), disp= True)

Let's write matched $\theta$:

In [ ]:
print( 'Wartość log wiarygodnosci  dla optymalnych parametrów: '+
      str(funkcjaLogWiarygodnosci(theta_opt, XX, y)))
print( 'theta: '+str(theta_opt))

## Results
We can receive logistic regression results in two ways:
* calculate the hypothesis value for the tested input and the adjusted parameters: this measure has the interpretation of the probability of belonging to the class 1,
* add the function performing the classification, i.e. comparison of the hypothesis value with 1/2:
   * for the value of the hypothesis> 1/2, the classification returns 1,
   * otherwise 0.

In [ ]:
def klasyfikacja(testX, theta):
    ''' Ta funkcja zwraca wynik klasyfikacji przykładu testX przy parametrach theta.
    Po obliczeniu hipotezy, jeśli otrzymane prawdopodobieństwo jest większe niż 0.5 to 
    zwraca 1 w przeciwnym wypadku zwraca 0'''
    h = ...
    if h > ... :
        klasa = ...
    else:
        klasa = ...
        
    return klasa

## Prediction
After adjusting the parameters, it's time to make a prediction.
Let's calculate the probability of acceptance of the candidate with the results
* 20 in mathematics
* 80 from biology
To predict, we use the hypothesis function, because according to our interpretation it gives the probability of acceptance

In [ ]:
prob = hipoteza([1, 20, 80], theta_opt)
print('dla kandydata z wymnikami 20 z matematyki i 80 z biologii prawdopodobieństwo przyjęcia wynosi: ' +str(prob))

Używając funkcji <tt>klasyfikacja</tt> dostaniemy klasę:
    

In [ ]:
klasa = klasyfikacja([1, 20, 80], theta_opt)
print('kandydat zalicza się do klasy: ' +str(klasa))

Let's draw the obtained division. Against the background of points colored in accordance with belonging to classes, we draw a simple separating area "1" from "0". It has the equation

$\qquad $ $ h_ theta (x) = 1/2 $,

ie:

$ \qquad $ $ \theta ^ T x = 0 $

or

$ \theta_0 + \theta_1 x_1 + \theta_2 x_2 = 0 $

Converting this to the straight equation in the coordinates $ (x_1, x_2) $ we have:

$ - \theta_2 x_2 = \theta_0 + \theta_1 x_1 $

$ x_2 = - \frac {1} {\theta_2} (\theta_0 + \theta_1 x_1) $

In [ ]:
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set3)
plt.xlabel('wynik z matematyki')
plt.ylabel('wynik z biologii')

x_plot = np.array([np.min(X[:,1]), np.max(X[:,1])])
y_plot = -1./theta_opt[2]*(theta_opt[1]*x_plot + theta_opt[0])

plt.plot(x_plot,y_plot,'b')

## Validation
The theory for this part can be found here:

https://brain.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wykład_Ocena_jakości_klasyfikacji

### Application in our example
We will now add a leave-one-out cross-validation.
In turn, we will postpone one example from the training set, and on such a reduced set we will learn the regression, and then we will check which of the following possible situations occurs:
* TP: the actual state is positive (y = 1) and the classifier is correct (result = 1)
* TN: the actual state is negative (y = 0) and the classifier is correct (result = 0)
* FP: false positives (false alarm): the actual state is negative (y = 0) but the classifier is wrong (result = 1)
* FN: missed alarm: the actual state is positive (y = 1) and the classifier is wrong (result = 0)

Przygotowujemy liczniki:

In [ ]:
TP = 0 
TN = 0
FP = 0 
FN = 0

 W pętli odkładamy przykład `v` do testowania:

In [ ]:
for v in range(len(y)):
    testX = XX[v]
    testY = y[v]
    # robimy zredukowany zbiór uczący przez usunięcie przykładu v
    uczX = np.delete(XX,v,axis=0)
    uczY = np.delete(y,v)
    # uczymy regresję na zredukowanym zbiorze uczącym
    theta_opt = so.fmin_bfgs(minusFunkcjaLogWiarygodnosci, theta0,
                             fprime=minusPochodnaLogWiarygodnosci, 
                             args=(..., ...), disp= False)
   # klasyfikujemy odłożony przykład : proszę uzupełnić funkcję klasyfikacja na początku pliku
    wynik = klasyfikacja(... , ...)
    # aktualizujemy liczniki; proszę uzupełnić kod:
    if testY == 1 and wynik == 1:
            ... += 1
    if testY == 1 and wynik == 0:
            ... +=1           
    if testY == 0 and wynik == 1:
            ... +=1
    if testY == 0 and wynik == 0:
            ... +=1
print('TP: ', TP)
print('FP: ', FP)
print('TN: ', TN)
print('FN: ', FN)

For our training set, we should get:
```
TP:  55
FP:  6
TN:  34
FN:  5
```

## ROC curve

To plot a ROC curve, a classification should be made for a number of possible threshold values for the hypothesis above which we consider the case to be in Class 1.


We modify the classification function so that the result also depends on the threshold:

In [ ]:
def klasyfikacjaProg(testX, theta,prog):
    h = hipoteza(testX, theta)
    if h > prog:
        klasa = ...
    else:
        klasa = ...
    return klasa

We can use this function to calculate the number of individual classification cases depending on the threshold:

In [ ]:
def liczROC(XX,y,progi):
    '''funkcja oblicza FPR i TPR dla zadanych progów,
       progi dla których mają być wyliczone wyniki podajemy w postaci wektora
       '''
    TP = np.zeros(len(progi))
    TN = np.zeros(len(progi))
    FP = np.zeros(len(progi))
    FN = np.zeros(len(progi))
    f = FloatProgress(min=0, max=len(y))
    display(f)
    for v in range(len(y)):
        f.value+=1
        
        testX = XX[v]
        testY = y[v]
        tenX = np.delete(XX,v,axis=0)
        tenY = np.delete(y,v)
        theta_opt = so.fmin_bfgs(minusFunkcjaLogWiarygodnosci, theta0, 
                                 fprime=minusPochodnaLogWiarygodnosci, 
                                 args=(tenX,tenY), disp= False)
        for ind, prog in enumerate(progi):
            wynik = klasyfikacjaProg(testX, theta_opt,prog)
           #==========================
           #      tu wstaw odpowiedni kawałek kodu
           #==========================
            ...
            
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN) 
    return (FPR,TPR)


You can use the following code to plot the ROC curve. In the graph, we mark the threshold values for which specific FPR and TPR values have been achieved.

In [ ]:
progi = np.arange(0.0,1.1,0.1)
FPR,TPR= liczROC(XX,y,progi)
plt.figure()
plt.plot(FPR,TPR,'o')
plt.plot(FPR,TPR)
for ind,pr in enumerate(progi):
    plt.text(FPR[ind],TPR[ind],str(pr))
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.xlim((0,1))
plt.ylim((0,1))
plt.show()

Let's count the field under the ROC curve, so-called AUC (area under curve), using the trapezoidal method:

In [ ]:
AUC = 0
for ind in range(len(progi)-1):
    a = ...
    b = ...
    h = ...
    AUC += 0.5*(a+b)*h
print('AUC: ',AUC)

We expect the result:
```
AUC:  0.9375
```